# Data Preprocessing 

1. (nickname + url) 기준 중복 제거 및 데이터 통합
2. 지정한 location의 review만 필터링
3. 수집한 restaurant의 review만 필터링 및 restaurant_id 업데이트
4. user_id 업데이트
5. 동일한 (user, restaurant) 쌍 review 중복 제거 (재방문 제거)
6. Review 수가 설정한 threshold 이상인 user만 필터링
7. utility matrix

In [ ]:
import pandas as pd
import os

### 1. (nickname + url) 기준 중복 제거 및 데이터 통합

In [ ]:
folder_path = '/proj-rs/data/02_restaurants/user_review'
output_path ='/proj-rs/data/02_restaurants/user_review_excel'
file_list = [file for file in os.listdir(folder_path) if file.endswith('.xlsx')]

all_data = []

for file in file_list:
    file_path = os.path.join(folder_path, file)
    xls = pd.ExcelFile(file_path)
    for sheet in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet, header=0)
        df['filename'] = file
        all_data.append(df)

# 데이터 통합
combined_df = pd.concat(all_data, ignore_index=True)

# 같은 (nickname + url)을 가진 것 중 가장 많이 등장한 store_id만 남기기
most_common_store = (
    combined_df.groupby(['nickname', 'url'])['store_id']
    .agg(lambda x: x.value_counts().idxmax())
    .reset_index()
)

# 중복으로 제거된 nickname 출력
original_triples = combined_df[['nickname', 'url', 'store_id']].drop_duplicates()
kept_triples = most_common_store[['nickname', 'url', 'store_id']]
dropped = pd.merge(original_triples, kept_triples, on=['nickname', 'url', 'store_id'], how='outer', indicator=True)
dropped = dropped[dropped['_merge'] == 'left_only']

if not dropped.empty:
    print("중복 제거:")
    for name in dropped['nickname'].unique():
        print(name)
else:
    print("중복 제거 X")

dedup_df = pd.merge(
    combined_df,
    most_common_store,
    on=['nickname', 'url', 'store_id'],
    how='inner'
)

dedup_df['url'] = dedup_df['url'].astype(str)

output_path = os.path.join(output_path, '05_1_reviews_combine.xlsx')
dedup_df.to_excel(output_path, index=False, engine='openpyxl')


### 2. 지정한 location의 review만 필터링
 - 홍제동
 - 연희동
 - 대현동
 - 창천동

In [ ]:
df = pd.read_excel("/proj-rs/data/02_restaurants/user_review_excel/05_1_reviews_combine.xlsx")

target_dongs = ['홍제동', '연희동', '대현동', '창천동']
filtered_df = df[df['store_location'].str.contains('|'.join(target_dongs))]

output_path ='/proj-rs/data/02_restaurants/user_review_excel'
output_path = os.path.join(output_path, '05_2_reviews_combine_location.xlsx')
filtered_df.to_excel(output_path, index=False, engine='openpyxl')

### 3. 수집한 restaurant의 review만 필터링 및 restaurant_id 업데이트

#### 1. restaurant_id 있음: Data 수집한 가게만 (실제 추천 시스템에 활용)

In [ ]:
restaurant_df = pd.read_json("/proj-rs/data/restaurant_info.json")
excel_df = pd.read_excel("/proj-rs/data/02_restaurants/user_review_excel/05_2_reviews_combine_location.xlsx")


merged_df = pd.merge(excel_df, restaurant_df[['restaurant_id', 'res_name']], 
                     left_on='store_name', right_on='res_name', how='left')

merged_df.dropna(subset=['restaurant_id'], inplace=True) # restaurant_id가 NaN인 행 제거
merged_df.drop(columns=['res_name', 'filename', 'store_id'], inplace=True)

output_path ='/proj-rs/data/02_restaurants/user_review_excel'
output_path = os.path.join(output_path, '05_3_reviews_combine_restaurant_id_nan_X.xlsx')
merged_df.to_excel(output_path, index=False, engine='openpyxl')

#### 2. restaurant_id 없음(NaN) 포함: 리뷰로만 존재하는 가게 포함 (확인 용도, 실제 추천 시스템에 활용 X)

In [ ]:
restaurant_df = pd.read_json("/proj-rs/data/restaurant_info.json")
excel_df = pd.read_excel("/proj-rs/data/02_restaurants/user_review_excel/05_2_reviews_combine_location.xlsx")


merged_df = pd.merge(excel_df, restaurant_df[['restaurant_id', 'res_name']], 
                     left_on='store_name', right_on='res_name', how='left')

merged_df.drop(columns=['res_name', 'filename', 'store_id'], inplace=True)

output_path ='/proj-rs/data/02_restaurants/user_review_excel'
output_path = os.path.join(output_path, '05_3_reviews_combine_restaurant_id_nan_O.xlsx')
merged_df.to_excel(output_path, index=False, engine='openpyxl')

### 4. user_id 업데이트

In [ ]:
input_path = "/proj-rs/data/02_restaurants/user_review_excel/05_3_reviews_combine_restaurant_id_nan_X.xlsx"
df = pd.read_excel(input_path)

df['user_id'] = df.groupby(['nickname', 'url'], sort=False).ngroup() + 1

output_path ='/proj-rs/data/02_restaurants/user_review_excel'
output_path = os.path.join(output_path, '05_4_reviews_combine_user_id_nan_X.xlsx')
df.to_excel(output_path, index=False, engine='openpyxl')

#### Review 수가 설정한 threshold 이상인 user 수 (재방문 제거 전)

In [ ]:
df = pd.read_excel("/proj-rs/data/02_restaurants/user_review_excel/05_4_reviews_combine_user_id_nan_X.xlsx") 

user_counts = df['user_id'].value_counts()

count_distribution = user_counts.value_counts().sort_index()
count_distribution = count_distribution.reset_index(name='num_users')
count_distribution.columns = ['review_count', 'num_users']

print(count_distribution)

    review_count  num_users
0              1      15385
1              2       4351
2              3       1967
3              4       1151
4              5        712
..           ...        ...
62            85          2
63            95          1
64           109          1
65           133          1
66           166          1

[67 rows x 2 columns]


In [6]:
for i in range(2, 10):
    print(f"threshold {count_distribution['review_count'][i]}: ", sum(count_distribution['num_users'][:i]), sum(count_distribution['num_users'][i:]))

threshold 3:  19736 6093
threshold 4:  21703 4126
threshold 5:  22854 2975
threshold 6:  23566 2263
threshold 7:  24063 1766
threshold 8:  24440 1389
threshold 9:  24704 1125
threshold 10:  24891 938


### 5. 동일한 (user, restaurant) 쌍 review 중복 제거 (재방문 제거)

In [ ]:
df = pd.read_excel("/proj-rs/data/02_restaurants/user_review_excel/05_4_reviews_combine_user_id_nan_X.xlsx")

df = df.sort_values('num_visit', ascending=False).drop_duplicates(subset=['user_id', 'store_name', 'url'], keep='first')

output_path ='/proj-rs/data/02_restaurants/user_review_excel'
output_path = os.path.join(output_path, '05_5_reviews_combine_nan_X_revisit.xlsx')
df.to_excel(output_path, index=False, engine='openpyxl')


#### Review 수가 설정한 threshold 이상인 user 수 (재방문 제거 후)

In [ ]:
df = pd.read_excel("/proj-rs/data/02_restaurants/user_review_excel/05_5_reviews_combine_nan_X_revisit.xlsx") 

user_counts = df['user_id'].value_counts()

count_distribution = user_counts.value_counts().sort_index()
count_distribution = count_distribution.reset_index(name='num_users')
count_distribution.columns = ['review_count', 'num_users']

for i in range(2, 10):
    print(f"threshold {count_distribution['review_count'][i]}: ", sum(count_distribution['num_users'][:i]), sum(count_distribution['num_users'][i:]))

threshold 3:  21181 4648
threshold 4:  23111 2718
threshold 5:  24126 1703
threshold 6:  24688 1141
threshold 7:  25055 774
threshold 8:  25300 529
threshold 9:  25428 401
threshold 10:  25558 271


In [ ]:
df = pd.read_excel("/proj-rs/data/02_restaurants/user_review_excel/05_3_reviews_combine_restaurant_id_nan_O.xlsx")

nan_restaurants = df[df['restaurant_id'].isna()]

nan_counts = nan_restaurants['store_name'].value_counts().reset_index()
nan_counts.columns = ['store_name', 'nan_count']

print(nan_counts)

                store_name  nan_count
0                      사러가       1825
1                스타벅스 홍제역점       1644
2                 버거킹 홍제역점       1584
3                맥도날드 홍제역점       1572
4             연희사러가 민영 주차장       1502
...                    ...        ...
3179      AMPM워시큐 연희브라운스톤점          1
3180              코맥스 아이앤디          1
3181                맛있는패밀리          1
3182                라온한식카페          1
3183  서대문구 신촌동 주민센터 전기차충전소          1

[3184 rows x 2 columns]


#### data 추가 수집을 위해 NaN값을 많이 가진 restaurant 확인

In [ ]:
df = pd.read_excel("/proj-rs/data/02_restaurants/user_review_excel/05_3_reviews_combine_restaurant_id_nan_O.xlsx")

nan_restaurants = df[df['restaurant_id'].isna()]

nan_counts = nan_restaurants['store_name'].value_counts().reset_index()
nan_counts.columns = ['store_name', 'nan_count']

filtered_nan_counts = nan_counts[nan_counts['nan_count'] >= 10]

total_nan_count = len(nan_restaurants)
print(f"총 nan 개수: {total_nan_count}\n")

print(filtered_nan_counts)

총 nan 개수: 159213

        store_name  nan_count
0              사러가       1825
1        스타벅스 홍제역점       1644
2         버거킹 홍제역점       1584
3        맥도날드 홍제역점       1572
4     연희사러가 민영 주차장       1502
...            ...        ...
1732    골드스윙 스크린골프         10
1733       에이스수능영어         10
1734  스카이수제고로케 홍제점         10
1735     연희MTA태권도장         10
1736          작은카페         10

[1737 rows x 2 columns]


In [ ]:
df = pd.read_excel("/proj-rs/data/02_restaurants/user_review_excel/05_3_reviews_combine_restaurant_id_nan_O.xlsx")

# restaurant_id가 NaN인 데이터
nan_restaurants = df[df['restaurant_id'].isna()]

# NaN store 개수 세기
nan_counts = nan_restaurants['store_name'].value_counts().reset_index()
nan_counts.columns = ['store_name', 'nan_count']
store_categories = nan_restaurants[['store_name', 'category']].drop_duplicates(subset='store_name')
nan_counts_with_cat = pd.merge(nan_counts, store_categories, on='store_name', how='left')

# 정렬 후 상위 N개
top_n = 100
top_nan_counts = nan_counts_with_cat.sort_values(by='nan_count', ascending=False).head(top_n)

total_nan_count = len(nan_restaurants)
print(f"총 nan 개수: {total_nan_count}\n")
print(top_nan_counts)

top_nan_counts.to_csv('/proj-rs/top_nan.csv', index=False, encoding='utf-8-sig')

총 nan 개수: 159213

      store_name  nan_count category
0            사러가       1825       시장
1      스타벅스 홍제역점       1644       카페
2       버거킹 홍제역점       1584      햄버거
3      맥도날드 홍제역점       1572      햄버거
4   연희사러가 민영 주차장       1502    민영주차장
..           ...        ...      ...
96       리정원 신촌점        301  육류,고기요리
95        설빙 신촌점        301       빙수
98      이공커피 연희점        296   카페,디저트
97     춘리마라탕 신촌점        296      마라탕
99    육회바른연어 홍대점        295     요리주점

[100 rows x 3 columns]


### 6. Review 수가 설정한 threshold 이상인 user만 필터링

In [ ]:
df = pd.read_excel("/proj-rs/data/02_restaurants/user_review_excel/05_5_reviews_combine_nan_X_revisit.xlsx")

# 사용자별 리뷰 수 계산
user_counts = df['user_id'].value_counts()

# 리뷰 수 threshold 이상인 사용자만 필터링
threshold = 5
users_to_keep = user_counts[user_counts >= threshold].index
filtered_df = df[df['user_id'].isin(users_to_keep)]

filtered_df.to_excel(f"/proj-rs/data/02_restaurants/user_review_excel/05_6_reviews_threshold{threshold}.xlsx", index=False)

In [ ]:
print(f"리뷰 수 {threshold} 이상 사용자 수: {len(users_to_keep)}명")
print(f"필터링 후 리뷰 수: {len(filtered_df)}개")
print(f"원래 리뷰 수: {len(df)}개")
# threshold 5:  24126 1703 1703명의 유저

리뷰 수 5 이상 사용자 수: 1703명
필터링 후 리뷰 수: 12405개
원래 리뷰 수: 47807개


### 7. utility matrix
- row: user
- column : restaurant 
- rating: number of visits
    - 0: 방문하지 않음
    - 1: 1번 방문함
    - 2: 2번 이상 방문함


In [ ]:
ut_df = pd.read_excel("/proj-rs/data/02_restaurants/user_review_excel/05_6_reviews_threshold5.xlsx")

users = ut_df['user_id'].unique()
restaurants = ut_df['restaurant_id'].unique()

# 모든 user_id x restaurant_id 조합
full_index = pd.MultiIndex.from_product([users, restaurants], names=['user_id', 'restaurant_id'])
full_df = pd.DataFrame(index=full_index).reset_index()

merged_df = pd.merge(full_df, ut_df[['user_id', 'restaurant_id', 'num_visit']], 
                     on=['user_id', 'restaurant_id'], how='left')

def visit_level(x):
    if pd.isna(x):
        return 0
    elif x == 1:
        return 1
    else:
        return 2

merged_df['visit_level'] = merged_df['num_visit'].apply(visit_level)
utility_matrix = merged_df.pivot(index='user_id', columns='restaurant_id', values='visit_level')

print(utility_matrix.head())

restaurant_id  1     3     7     25    42    56    65    67    95    101   \
user_id                                                                     
3                 0     0     0     0     0     0     0     0     0     0   
26                0     0     0     0     0     0     0     0     0     0   
47                0     0     0     0     0     0     0     0     0     0   
55                0     0     0     0     0     0     0     0     0     0   
64                0     0     0     0     0     0     0     0     0     0   

restaurant_id  ...  3594  3597  3602  3607  3617  3622  3630  3633  3634  3635  
user_id        ...                                                              
3              ...     0     1     0     0     0     0     0     0     0     0  
26             ...     0     0     0     0     0     0     0     0     0     0  
47             ...     0     0     0     0     0     0     0     0     0     0  
55             ...     0     0     0     0     0     0 

In [ ]:
utility_matrix.to_excel("/proj-rs/data/02_restaurants/user_review_excel/05_7_utility_matrix.xlsx")